In [1]:
import re
import os
import nltk
import tensorflow as tf
import numpy as np
import pandas as pd
from nltk.corpus import reuters
import collections
from transformers import pipeline

%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
documents = reuters.fileids()
train_docs = list(filter(lambda doc: doc.startswith("train"), documents))
test_docs = list(filter(lambda doc: doc.startswith("test"), documents))
categories = reuters.categories()

<IPython.core.display.Javascript object>

In [3]:
title_re = "([A-Z].*[a-z]$)"


def title_split(doc):
    prev_word = ""
    for word in doc:
        if bool(re.match(title_re, word)):
            title_pos = doc.index(word)
            if prev_word == "A" or word == "USA":
                title_pos = title_pos - 1
            return " ".join(doc[:title_pos])
        prev_word = word


def words_split(doc):
    prev_word = ""
    for word in doc:
        if bool(re.match(title_re, word)):
            title_pos = doc.index(word)
            if prev_word == "A" or word == "USA":
                title_pos = title_pos - 1
            return " ".join(doc[title_pos:])
        prev_word = word

<IPython.core.display.Javascript object>

In [4]:
train_list = [int(x.split("/")[1]) for x in train_docs]
cat_list = [reuters.categories(x) for x in train_docs]
title_list = [title_split(reuters.words(x)) for x in train_docs]
words_list = [words_split(reuters.words(x)) for x in train_docs]

<IPython.core.display.Javascript object>

In [5]:
news_df = pd.DataFrame(
    {
        "DocID": train_list,
        "Categories": cat_list,
        "Title": title_list,
        "Content": words_list,
    }
).sort_values(by="DocID")

<IPython.core.display.Javascript object>

In [6]:
def title_split(row):
    split_word = row["Content"].split()[0]
    title_new = split_word + row["Content"].split(split_word)[1]
    row["Title"] = title_new
    row["Content"] = title_new
    return row

<IPython.core.display.Javascript object>

In [7]:
title_split_df = news_df[news_df["Title"] == ""].apply(title_split, axis=1)
cols = list(news_df.columns)
news_df.loc[news_df.DocID.isin(title_split_df.DocID), cols] = title_split_df[cols]

<IPython.core.display.Javascript object>

In [8]:
cat_dict = {
    "acq": "acuisition",
    "alu": "aluminum",
    "bop": "bottom of the pyramid",
    "castor-oil": "castor oil",
    "carcass": "livestock",
    "cpu": "computershare",
    "coconut-oil": "coconut oil",
    "copra-cake": "copra cake",
    "cpi": "consumer price index",
    "dfl": "deutsche fussball liga",
    "dlr": "digital realty trust",
    "dmk": "dravida munnetra kazhagam",
    "earn": "ellington residential mortgage reit stock",
    "gnp": "grupo nacional provincial",
    "groundnut-oil": "ground nut oil",
    "instal-debt": "instalco intressenter debt",
    "ipi": "intrepid potash",
    "iron-steel": "iron steel",
    "l-cattle": "cattle",
    "lei": "leading economic index",
    "lin-oil": "linseed oil",
    "meal-feed": "meal feed",
    "money-fx": "money foreign exchange",
    "money-supply": "money supply",
    "nat-gas": "natural gas",
    "nkr": "nokia",
    "palm-oil": "palm oil",
    "palmkernel": "palm kernel",
    "pet-chem": "petronas chemicals",
    "rape-oil": "rapeseed oil",
    "soy-meal": "soy meal",
    "soy-oil": "soy oil",
    "strategic-metal": "strategic metal",
    "sun-meal": "sunflower meal",
    "sun-oil": "sunflower oil",
    "sunseed": "sunflower seed",
    "veg-oil": "vagetable oil",
    "wpi": "waterfront philippines",
}

<IPython.core.display.Javascript object>

In [9]:
new_cats = [
    "acuisition",
    "aluminum",
    "bottom of the pyramid",
    "barley",
    "castor oil",
    "carcass",
    "cattle",
    "cocoa",
    "coconut",
    "coconut oil",
    "coffee",
    "consumer price index",
    "copper",
    "copra cake",
    "corn",
    "cotton",
    "computershare",
    "crude",
    "deutsche fussball liga",
    "digital realty trust",
    "dravida munnetra kazhagam",
    "ellington residential mortgage reit stock",
    "fuel",
    "gas",
    "gold",
    "grain",
    "groundnut",
    "ground nut oil",
    "grupo nacional provincial",
    "heat",
    "hog",
    "housing",
    "income",
    "instalco intressenter debt",
    "interest",
    "intrepid potash",
    "iron steel",
    "jet",
    "jobs",
    "lead",
    "leading economic index",
    "linseed oil",
    "livestock",
    "lumber",
    "meal feed",
    "money foreign exchange",
    "money supply",
    "natural gas",
    "naphtha",
    "nickel",
    "nokia",
    "new zealand dollar",
    "oat",
    "orange",
    "palladium",
    "palm oil",
    "palm kernel",
    "petronas chemicals",
    "platinum",
    "potato",
    "propane",
    "rand",
    "rapeseed",
    "rapeseed oil",
    "reserves",
    "retail",
    "rice",
    "rubber",
    "rye",
    "ship",
    "silver",
    "sorghum",
    "soybean",
    "soy meal",
    "soy oil",
    "strategic metal",
    "sugar",
    "sunflower meal",
    "sunflower oil",
    "sunflower seed",
    "tea",
    "tin",
    "trade",
    "vagetable oil",
    "waterfront philippines",
    "wheat",
    "yen",
    "zinc",
]

<IPython.core.display.Javascript object>

In [10]:
classifier = pipeline("zero-shot-classification", model="roberta-large-mnli")

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at roberta-large-mnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


<IPython.core.display.Javascript object>

In [17]:
news_df["top_label"] = news_df["Categories"].apply(lambda x: x[0])
news_df["title_length"] = news_df["Title"].apply(lambda x: len(x) if x != None else 0)
news_df["con_length"] = news_df["Content"].apply(lambda x: len(x) if x != None else 0)

<IPython.core.display.Javascript object>

In [20]:
news_df["con_length"].median()

485.0

<IPython.core.display.Javascript object>

In [12]:
news_filt_df = news_df[news_df["title_length"] > 40]
sample_df = (
    news_filt_df.groupby("top_label")
    .apply(lambda x: x.sample(15, replace=True))
    .reset_index(drop=True)
)
sample_df = sample_df.drop(columns=["Categories"]).drop_duplicates()

<IPython.core.display.Javascript object>

In [13]:
class_title_list = list(sample_df["Title"])

<IPython.core.display.Javascript object>

In [14]:
clfd_dict_list2 = [
    classifier(title, new_cats, multi_label=True) for title in class_title_list
]
categorized_df = pd.DataFrame.from_records(clfd_dict_list2)
categorized_df.to_pickle("C:/Users/rparg/Documents/Data/Reuters/categorized2.pkl")

<IPython.core.display.Javascript object>

In [22]:
news_con_filt_df = news_df[
    (news_df["con_length"] > 200) & (news_df["con_length"] < 1000)
]

<IPython.core.display.Javascript object>

In [27]:
sample_con_df = (
    news_con_filt_df.groupby("top_label")
    .apply(lambda x: x.sample(4, replace=True))
    .reset_index(drop=True)
)
sample_con_df = sample_con_df.drop(columns=["Categories"]).drop_duplicates()

<IPython.core.display.Javascript object>

In [31]:
class_con_list = list(sample_con_df["Content"])

<IPython.core.display.Javascript object>

In [32]:
class_con_list

["Waste Management Inc said it amended its offer to buy the outstanding shares of ChemLawn Corp to 35 dlrs a share . On Thursday the company said it was prepared to bid 33 dlrs a share , up from its original 27 dlrs a share offer . As a result of the price increase , made through Waste Management ' s wholly owned subsidiary , WMX Acquisition Corp , the offer has been extended and the withdrawal rights will not expire at midnight EDT April Two , unless further extended . Waste Management also said it amended its offer to provide that the tender offer price will not be reduced by ChemLawn ' s 10 cts a share dividend payable to holders of record April 1 , 1987 .",
 'An investor group including New York - based Mutual Shares Corp and Mutual Qualified Income Fund Inc told the Securities and Exchange Commission it raised its stake in Gelco Corp common stock to 575 , 859 shares or 8 . 2 pct of the total outstanding . The group said its most recent purchases included 241 , 000 shares bought be

<IPython.core.display.Javascript object>

In [28]:
clfd_dict_list2 = [
    classifier(title, new_cats, multi_label=True) for title in class_title_list
]
categorized_df = pd.DataFrame.from_records(clfd_dict_list2)
categorized_df.to_pickle("C:/Users/rparg/Documents/Data/Reuters/categorized2.pkl")

,DocID,Title,Content,top_label,title_length,con_length
0,7789,WASTE MANAGEMENT & lt ; WMX > HIKES CHEMLAWN &...,Waste Management Inc said it amended its offer...,acq,62,665
1,6103,GROUP BOOSTS GELCO & lt ; GEL > STAKE TO 8 . 2...,An investor group including New York - based M...,acq,50,424
2,3384,UNITED SECURITY & lt ; UNSE > TO BUY ROBERT BR...,United Security Financial Corp of Illinois sai...,acq,54,567
3,9064,MARYLAND NATIONAL & lt ; MDNT > SEES NEW NAME,"Maryland National Corp , the parent of Marylan...",acq,45,578
4,12484,LME DETAILS MARCH 1987 TURNOVER,"The London Metal Exchange , LME , issued turno...",alum,31,967
...,...,...,...,...,...,...
262,13232,SOUTH KOREAN WHOLESALE PRICES UP 0 . 2 PCT IN ...,"South Korea ' s wholesale price index , base 1...",wpi,51,349
263,4641,SWISS WHOLESALE PRICES FALL 0 . 3 PCT IN FEBRUARY,Swiss wholesale prices fell by 0 . 3 pct in Fe...,wpi,49,533
264,3183,"HUDSON BAY MINING CUTS U . S ., CANADA ZINC PR...",Hudson Bay Mining and Smelting Co Ltd > said i...,zinc,57,610
266,3104,"U . S . MINT SEEKING 4 , 784 , 000 LBS ZINC",The U . S . Mint said it is seeking offers on ...,zinc,43,798


<IPython.core.display.Javascript object>